# Exercice 07 - Ingestion PostgreSQL vers Bronze

## Objectifs
- Extraire des donnees de PostgreSQL
- Sauvegarder dans la couche Bronze de MinIO
- Organiser les donnees par date d'ingestion
- Creer un pipeline d'ingestion reutilisable

---

## 1. Architecture d'ingestion

```
+----------------+                    +------------------------+
|                |                    |        MinIO           |
|   PostgreSQL   |     SPARK          |                        |
|                |  =============>    |  +------------------+  |
|  +----------+  |                    |  |     BRONZE       |  |
|  | customers|  |                    |  +------------------+  |
|  | products |  |                    |  | /customers/      |  |
|  | orders   |  |                    |  |   /2024-01-15/   |  |
|  | ...      |  |                    |  | /products/       |  |
|  +----------+  |                    |  |   /2024-01-15/   |  |
|                |                    |  | /orders/         |  |
+----------------+                    |  |   /2024-01-15/   |  |
                                      |  +------------------+  |
                                      +------------------------+

Format Bronze : donnees brutes en Parquet
Organisation  : /table/date_ingestion/
```

## 2. Configuration Spark pour PostgreSQL et MinIO

In [1]:
from pyspark.sql import SparkSession
from datetime import datetime

# Creer la SparkSession avec les configurations
spark = SparkSession.builder \
    .appName("Ingestion PostgreSQL vers Bronze") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0,org.apache.hadoop:hadoop-aws:3.4.1,com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

print("Spark configure pour PostgreSQL et MinIO")

Spark configure pour PostgreSQL et MinIO


In [2]:
# Configuration PostgreSQL
jdbc_url = "jdbc:postgresql://postgres:5432/app"
jdbc_properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

# Date d'ingestion pour organiser les fichiers
date_ingestion = datetime.now().strftime("%Y-%m-%d")
print(f"Date d'ingestion : {date_ingestion}")

Date d'ingestion : 2026-01-14


## 3. Ingerer une table simple

In [3]:
# Lire la table customers depuis PostgreSQL
df_customers = spark.read.jdbc(
    url=jdbc_url,
    table="customers",
    properties=jdbc_properties
)

print(f"Customers : {df_customers.count()} lignes")
df_customers.show(5)

Customers : 91 lignes
+-----------+--------------------+------------------+--------------------+--------------------+-----------+------+-----------+-------+--------------+--------------+
|customer_id|        company_name|      contact_name|       contact_title|             address|       city|region|postal_code|country|         phone|           fax|
+-----------+--------------------+------------------+--------------------+--------------------+-----------+------+-----------+-------+--------------+--------------+
|      ALFKI| Alfreds Futterkiste|      Maria Anders|Sales Representative|       Obere Str. 57|     Berlin|  NULL|      12209|Germany|   030-0074321|   030-0076545|
|      ANATR|Ana Trujillo Empa...|      Ana Trujillo|               Owner|Avda. de la Const...|México D.F.|  NULL|      05021| Mexico|  (5) 555-4729|  (5) 555-3745|
|      ANTON|Antonio Moreno Ta...|    Antonio Moreno|               Owner|     Mataderos  2312|México D.F.|  NULL|      05023| Mexico|  (5) 555-3932|    

In [4]:
# Sauvegarder dans Bronze
chemin_bronze = f"s3a://bronze/customers/{date_ingestion}"

df_customers.write \
    .mode("overwrite") \
    .parquet(chemin_bronze)

print(f"Sauvegarde reussie : {chemin_bronze}")

Sauvegarde reussie : s3a://bronze/customers/2026-01-14


In [5]:
# Verifier la sauvegarde
df_check = spark.read.parquet(chemin_bronze)
print(f"Verification : {df_check.count()} lignes lues depuis Bronze")

Verification : 91 lignes lues depuis Bronze


## 4. Fonction d'ingestion reutilisable

In [6]:
def ingerer_table(nom_table, date=None):
    """
    Ingere une table PostgreSQL vers le bucket Bronze.
    
    Args:
        nom_table: Nom de la table a ingerer
        date: Date d'ingestion (defaut: aujourd'hui)
    
    Returns:
        Nombre de lignes ingerees
    """
    if date is None:
        date = datetime.now().strftime("%Y-%m-%d")
    
    # Lire depuis PostgreSQL
    df = spark.read.jdbc(
        url="jdbc:postgresql://postgres:5432/app",
        table=nom_table,
        properties={
            "user": "postgres",
            "password": "postgres",
            "driver": "org.postgresql.Driver"
        }
    )
    
    # Sauvegarder dans Bronze
    chemin = f"s3a://bronze/{nom_table}/{date}"
    df.write.mode("overwrite").parquet(chemin)
    
    nb_lignes = df.count()
    print(f"[OK] {nom_table} : {nb_lignes} lignes -> {chemin}")
    
    return nb_lignes

In [7]:
# Tester la fonction
ingerer_table("products")

[OK] products : 77 lignes -> s3a://bronze/products/2026-01-14


77

## 5. Ingerer toutes les tables Northwind

In [8]:
# Liste des tables a ingerer
tables_northwind = [
    "categories",
    "customers",
    "employees",
    "orders",
    "order_details",
    "products",
    "shippers",
    "suppliers",
    "region",
    "territories"
]

print(f"Tables a ingerer : {len(tables_northwind)}")

Tables a ingerer : 10


In [9]:
# Ingerer toutes les tables
resultats = {}

print("=" * 50)
print("INGESTION NORTHWIND VERS BRONZE")
print("=" * 50)

for table in tables_northwind:
    try:
        nb_lignes = ingerer_table(table, date_ingestion)
        resultats[table] = {"status": "OK", "lignes": nb_lignes}
    except Exception as e:
        print(f"[ERREUR] {table} : {e}")
        resultats[table] = {"status": "ERREUR", "erreur": str(e)}

print("=" * 50)
print("INGESTION TERMINEE")
print("=" * 50)

INGESTION NORTHWIND VERS BRONZE
[OK] categories : 8 lignes -> s3a://bronze/categories/2026-01-14
[OK] customers : 91 lignes -> s3a://bronze/customers/2026-01-14
[OK] employees : 9 lignes -> s3a://bronze/employees/2026-01-14
[OK] orders : 830 lignes -> s3a://bronze/orders/2026-01-14
[OK] order_details : 2155 lignes -> s3a://bronze/order_details/2026-01-14
[OK] products : 77 lignes -> s3a://bronze/products/2026-01-14
[OK] shippers : 6 lignes -> s3a://bronze/shippers/2026-01-14
[OK] suppliers : 29 lignes -> s3a://bronze/suppliers/2026-01-14
[OK] region : 4 lignes -> s3a://bronze/region/2026-01-14
[OK] territories : 53 lignes -> s3a://bronze/territories/2026-01-14
INGESTION TERMINEE


In [10]:
# Resume de l'ingestion
print("\nResume :")
total_lignes = 0
for table, info in resultats.items():
    if info["status"] == "OK":
        print(f"  {table}: {info['lignes']} lignes")
        total_lignes += info["lignes"]
    else:
        print(f"  {table}: ERREUR")

print(f"\nTotal : {total_lignes} lignes ingerees")


Resume :
  categories: 8 lignes
  customers: 91 lignes
  employees: 9 lignes
  orders: 830 lignes
  order_details: 2155 lignes
  products: 77 lignes
  shippers: 6 lignes
  suppliers: 29 lignes
  region: 4 lignes
  territories: 53 lignes

Total : 3262 lignes ingerees


## 6. Ajouter des metadonnees d'ingestion

In [11]:
from pyspark.sql import functions as F

def ingerer_table_avec_metadata(nom_table, date=None):
    """
    Ingere une table avec des colonnes de metadata.
    """
    if date is None:
        date = datetime.now().strftime("%Y-%m-%d")
    
    timestamp_ingestion = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Lire depuis PostgreSQL
    df = spark.read.jdbc(
        url="jdbc:postgresql://postgres:5432/app",
        table=nom_table,
        properties={
            "user": "postgres",
            "password": "postgres",
            "driver": "org.postgresql.Driver"
        }
    )
    
    # Ajouter les metadonnees
    df = df.withColumn("_source", F.lit("postgresql")) \
           .withColumn("_table", F.lit(nom_table)) \
           .withColumn("_ingestion_date", F.lit(date)) \
           .withColumn("_ingestion_timestamp", F.lit(timestamp_ingestion))
    
    # Sauvegarder dans Bronze
    chemin = f"s3a://bronze/{nom_table}/{date}"
    df.write.mode("overwrite").parquet(chemin)
    
    nb_lignes = df.count()
    print(f"[OK] {nom_table} : {nb_lignes} lignes (avec metadata) -> {chemin}")
    
    return nb_lignes

In [12]:
# Tester avec metadata
ingerer_table_avec_metadata("categories")

# Verifier les metadonnees
df_test = spark.read.parquet(f"s3a://bronze/categories/{date_ingestion}")
df_test.select("category_id", "category_name", "_source", "_table", "_ingestion_date").show()

[OK] categories : 8 lignes (avec metadata) -> s3a://bronze/categories/2026-01-14
+-----------+--------------+----------+----------+---------------+
|category_id| category_name|   _source|    _table|_ingestion_date|
+-----------+--------------+----------+----------+---------------+
|          1|     Beverages|postgresql|categories|     2026-01-14|
|          2|    Condiments|postgresql|categories|     2026-01-14|
|          3|   Confections|postgresql|categories|     2026-01-14|
|          4|Dairy Products|postgresql|categories|     2026-01-14|
|          5|Grains/Cereals|postgresql|categories|     2026-01-14|
|          6|  Meat/Poultry|postgresql|categories|     2026-01-14|
|          7|       Produce|postgresql|categories|     2026-01-14|
|          8|       Seafood|postgresql|categories|     2026-01-14|
+-----------+--------------+----------+----------+---------------+



## 7. Verifier le contenu de Bronze

In [13]:
# Lister les fichiers dans Bronze
print("Contenu du bucket Bronze :")
print("=" * 50)

for table in tables_northwind:
    try:
        chemin = f"s3a://bronze/{table}/{date_ingestion}"
        df = spark.read.parquet(chemin)
        print(f"{table}: {df.count()} lignes")
    except:
        print(f"{table}: non trouve")

Contenu du bucket Bronze :
categories: 8 lignes
customers: 91 lignes
employees: 9 lignes
orders: 830 lignes
order_details: 2155 lignes
products: 77 lignes
shippers: 6 lignes
suppliers: 29 lignes
region: 4 lignes
territories: 53 lignes


---

## Exercice

**Objectif** : Creer un script d'ingestion complet

**Consigne** :
1. Creez une fonction `ingestion_complete()` qui :
   - Ingere toutes les tables avec metadonnees
   - Affiche un rapport final
   - Retourne un dictionnaire avec les statistiques

2. Ajoutez une colonne `_nb_colonnes` aux metadonnees

A vous de jouer :

In [14]:
from pyspark.sql import functions as F
from datetime import datetime

def ingestion_complete(tables_liste, bucket_bronze="s3a://bronze"):
    """
    Ingère une liste de tables avec métadonnées complètes et rapport final.
    """
    date_jour = datetime.now().strftime("%Y-%m-%d")
    timestamp_now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    stats = {}
    
    print(f"=== DÉBUT INGESTION ({timestamp_now}) ===\n")

    for table in tables_liste:
        try:
            # 1. Lecture JDBC
            df = spark.read.jdbc(
                url="jdbc:postgresql://postgres:5432/app",
                table=table,
                properties={"user": "postgres", "password": "postgres", "driver": "org.postgresql.Driver"}
            )
            
            # Calculs préliminaires pour les métadonnées
            nb_lignes = df.count()
            nb_cols = len(df.columns)
            
            # 2. Ajout des métadonnées (dont _nb_colonnes demandé dans l'exercice)
            df_final = df.withColumn("_source", F.lit("postgresql")) \
                         .withColumn("_table", F.lit(table)) \
                         .withColumn("_ingestion_timestamp", F.lit(timestamp_now)) \
                         .withColumn("_nb_colonnes", F.lit(nb_cols)) # La consigne est ici
            
            # 3. Écriture dans Bronze (partitionné par date)
            chemin = f"{bucket_bronze}/{table}/{date_jour}"
            df_final.write.mode("overwrite").parquet(chemin)
            
            # Enregistrement du succès
            stats[table] = {"status": "OK", "lignes": nb_lignes, "colonnes": nb_cols}
            print(f"[SUCCÈS] {table:15} : {nb_lignes:5} lignes | {nb_cols:2} cols -> {chemin}")
            
        except Exception as e:
            stats[table] = {"status": "ERREUR", "msg": str(e)}
            print(f"[ERREUR] {table:15} : {str(e)}")

    # 4. Rapport Final
    print("\n=== RAPPORT FINAL ===")
    succes = sum(1 for t in stats.values() if t["status"] == "OK")
    print(f"Tables traitées avec succès : {succes} / {len(tables_liste)}")
    
    return stats

# Test de la fonction avec toutes les tables
tables_northwind = ["categories", "customers", "employees", "orders", "products"]
resultats = ingestion_complete(tables_northwind)

=== DÉBUT INGESTION (2026-01-14 16:41:40) ===

[SUCCÈS] categories      :     8 lignes |  4 cols -> s3a://bronze/categories/2026-01-14
[SUCCÈS] customers       :    91 lignes | 11 cols -> s3a://bronze/customers/2026-01-14
[SUCCÈS] employees       :     9 lignes | 18 cols -> s3a://bronze/employees/2026-01-14
[SUCCÈS] orders          :   830 lignes | 14 cols -> s3a://bronze/orders/2026-01-14
[SUCCÈS] products        :    77 lignes | 10 cols -> s3a://bronze/products/2026-01-14

=== RAPPORT FINAL ===
Tables traitées avec succès : 5 / 5


---

## Resume

Dans ce notebook, vous avez appris :
- Comment **extraire des donnees** de PostgreSQL avec Spark
- Comment **sauvegarder** les donnees dans MinIO (Bronze)
- Comment **organiser** les donnees par date d'ingestion
- Comment **ajouter des metadonnees** pour la tracabilite
- Comment creer un **pipeline d'ingestion** reutilisable

### Prochaine etape
Dans le prochain notebook, nous apprendrons a ingerer des donnees depuis le Web (API REST).